In [119]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

from scipy.stats import norm
from scipy.stats import multivariate_normal

from iminuit import Minuit
from iminuit.cost import UnbinnedNLL


In [120]:
size = 1000

In [121]:
#Construct the model

def coordinate_transform (data: np.array([[]])) -> np.array([[]]):
    """
    Input: data in the form of [r, theta, phi]
    Output: data in the form of [x+,x-,y+,y-]
    """
    data_xy = []
    for x in data: 
        data_xy.append([
            x[0]*math.cos(x[1]+x[2]),
            x[0]*math.cos(x[1]-x[2]),
            x[0]*math.sin(x[1]+x[2]),
            x[0]*math.sin(x[1]-x[2]),
        ])
    return np.array(data_xy)

def model(x, mu, cov):
    """
    Return the pdf of x(x+,x-,y+,y-) given a distribution in r space
    """
    # randomly generate points in r space
    data_r = np.random.multivariate_normal(mu, cov, size = 100000)
    # transform the data to xy space
    data_xy = coordinate_transform(data_r)
    # find the mean
    mu_x = np.mean(data_xy, axis = 0)
    cov_x = np.cov(data_xy, rowvar = False)
    print (mu_x, cov_x)
    return multivariate_normal.pdf(x, mu_x, cov_x)

In [140]:
### Construct the mean and covariance matrix
mean = np.array([-0.93,-1.3,5.7,6.5])
std = np.array([8.2, 8.4, 8.2, 8.3])
corr = np.array([[1, -0.1, -0.05, 0.1],
        [-0.1, 1, 0.1, -0.05],
        [-0.05, 0.1, 1, 0.1],
        [0.1, -0.05, 0.1, 1]])

cov = np.array([[corr[i, j] * std[i] * std[j] for i in range (0,4)] for j in range (0,4)])
print ("covaraince_Matrix in x_y space:\n", cov)


covaraince_Matrix in x_y space:
 [[67.24  -6.888 -3.362  6.806]
 [-6.888 70.56   6.888 -3.486]
 [-3.362  6.888 67.24   6.806]
 [ 6.806 -3.486  6.806 68.89 ]]


In [131]:
# generate the data
dset = np.random.multivariate_normal(mean, cov, size=size)
print (dset)

[[ -4.33646489   7.92730283  -3.94288802   4.16771799]
 [ 10.85904394  -6.00920589   6.0646319    0.22017544]
 [ -5.87289264  -9.42360022   6.99935419   2.81932443]
 ...
 [ -5.7611007   -1.02008086  19.10014035   9.59197131]
 [ -5.06838052 -11.60259794  14.31460686  16.02479242]
 [ 11.64847025  -0.70130865   8.90406135  11.11685647]]


In [132]:
# convert the data in r_space
dset_r = coordinate_transform(dset)
print (dset_r)

[[  2.88530452  -3.32735939   3.2372744    2.78093639]
 [ 10.84236849   9.56831645   0.60156543   5.1347985 ]
 [  4.425534     4.43461564   3.86076635  -3.85033143]
 ...
 [ -4.13797473  -1.70324286   4.00842194   5.50356658]
 [  4.60786342  -3.58723412  -2.11094193   3.58053522]
 [ -3.98078463 -11.45903732  10.94715547   2.09220528]]


In [136]:
# make the cost function, in this case the
 # negative log likelihood
nll = UnbinnedNLL(dset , model)


In [168]:
mean_r_guess = np.array([0,0,0])
cov_r_guess = np.array([[1,0,0],[0,1,0],[0,0,1]])
# s
print (mean_r_guess, cov_r_guess)
mi = Minuit(nll, N= 100, mu = mean_r_guess, cov = cov_r_guess) # pass in mu and cov in r space

[0 0 0] [[1 0 0]
 [0 1 0]
 [0 0 1]]


RuntimeError: N is not one of the parameters [mu cov]

In [166]:
mi.migrad()

ValueError: mean must be 1 dimensional

In [ ]:
mi.hesse ()